In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from typing import List
import transformers
from tokenizers import AddedToken
import torch
from torch import nn

In [2]:
class LinearWrapper(nn.Module):
    def __init__(self, layer: nn.Linear, num_embeddings: int):
        super().__init__()
        self.layer = layer
        self.num_embeddings = num_embeddings
        self.n_new_tokens = num_embeddings - layer.out_features
        self.new_embeddings = nn.Linear(layer.in_features, self.n_new_tokens)
        self.new_embeddings.to(layer.weight.device).to(layer.weight.dtype)
    
    def forward(self, x):
        z1 = self.layer(x)
        z2 = self.new_embeddings(x)
        return torch.cat([z1, z2], dim=-1)

class EmbeddingWrapper(nn.Module):
    def __init__(self, embedding: nn.Embedding, num_embeddings: int):
        super().__init__()
        self.embedding_dim = embedding.embedding_dim
        self.num_embeddings = num_embeddings
        self.n_new_tokens = num_embeddings - embedding.num_embeddings
        # inspired from here 
        # https://github.com/huggingface/transformers/blob/185463784e0a0b4cd7974ce5bded7a52ae170f6d/src/transformers/modeling_utils.py#L2026
        self.old_embeddings = nn.Embedding(self.num_embeddings, self.embedding_dim)
        self.old_embeddings.weight.data = torch.zeros_like(self.old_embeddings.weight.data)
        self.old_embeddings.weight.data[:embedding.num_embeddings] = embedding.weight.data
        self.old_embeddings.to(embedding.weight.device).to(embedding.weight.dtype)
        self.new_embeddings = nn.Embedding(self.num_embeddings, self.embedding_dim)
        self.new_embeddings.weight.data[:embedding.num_embeddings] = torch.zeros_like(embedding.weight.data)
        self.new_embeddings.to(embedding.weight.device).to(embedding.weight.dtype)

    
    def forward(self, x):
        return self.old_embeddings(x) + self.new_embeddings(x)

class Llama2EmbeddingSurgeon():
    def __init__(self, llama, extended_tokenizer):
        self.llama = llama 
        self.extended_tokenizer = extended_tokenizer
        self.extended_embedding = EmbeddingWrapper(model.model.embed_tokens, len(extended_tokenizer))
        self.extended_unembedding = LinearWrapper(model.lm_head, len(extended_tokenizer))
        
    def get_surgeried_llama(self):
        self.backup_embed_tokens = self.llama.model.embed_tokens
        self.backup_lm_head = self.llama.lm_head
        self.llama.model.embed_tokens = self.extended_embedding
        self.llama.lm_head = self.extended_unembedding
        return self.llama
    
    def save(self, llama, path):
        # check if llama is surgeried
        assert llama.model.embed_tokens == self.extended_embedding
        assert llama.lm_head == self.extended_unembedding
        self.llama.model.embed_tokens = self.backup_embed_tokens
        self.llama.lm_head = self.backup_lm_head
        self.llama.save_pretrained(path)
        self.extended_tokenizer.save_pretrained(path)
        torch.save(self.extended_embedding.state_dict(), f"{path}/extended_embedding.pt")
        torch.save(self.extended_unembedding.state_dict(), f"{path}/extended_unembedding.pt") 
    
    @classmethod
    def load(cls, path):
        extended_embedding_dict = torch.load(f"{path}/extended_embedding.pt")
        extended_unembedding_dict = torch.load(f"{path}/extended_unembedding.pt")
        llama = AutoModelForCausalLM.from_pretrained(path)
        tokenizer = AutoTokenizer.from_pretrained(path)
        surgeon = cls(llama, tokenizer)
        surgeon.extended_embedding.load_state_dict(extended_embedding_dict)
        surgeon.extended_unembedding.load_state_dict(extended_unembedding_dict)
        return surgeon

In [3]:
model_name_or_path = '/dlabscratch1/public/llm_weights/llama2_hf/Llama-2-7b-hf/'
# load model in torch.bfloat16
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, device_map='auto', torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
pause_token = AddedToken("<|pause|>", 
                         single_word=False, 
                         lstrip=True, 
                         rstrip=True)
                         #special=True, 
                         #normalized=False)

In [5]:
tokenizer.add_tokens([pause_token], special_tokens=True)
print(tokenizer)
# get idx of pause otken
pause_token_id = tokenizer.convert_tokens_to_ids("<|pause|>")
print(pause_token_id)

LlamaTokenizerFast(name_or_path='/dlabscratch1/public/llm_weights/llama2_hf/Llama-2-7b-hf/', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|pause|>", rstrip=True, lstrip=True, single_word=False, normalized=False, special=True),
}
32000


In [6]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head):

In [7]:
## conventionally you'd do this like this:
#model.resize_token_embeddings(len(tokenizer))
        
## ours
surgeon = Llama2EmbeddingSurgeon(model, tokenizer)
model = surgeon.get_surgeried_llama()
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): EmbeddingWrapper(
      (old_embeddings): Embedding(32001, 4096)
      (new_embeddings): Embedding(32001, 4096)
    )
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post

In [8]:
print(pause_token_id)

32000


In [9]:
# test
toks1 = tokenizer.encode('The<|pause|> quick<|pause|> brown <|pause|> fox jumps over the lazy dog', return_tensors='pt')
toks2 = tokenizer.encode('The quick brown fox jumps over the lazy dog', return_tensors='pt')
idx2 = 0
for idx1 in range(len(toks1[0])):
    if toks1[0, idx1].item() != pause_token_id:
        print('w/o', toks2[0, idx2].item(), tokenizer.decode([toks2[0, idx2]]), 'w', toks1[0, idx1].item(), tokenizer.decode([toks1[0, idx1]]))
        assert toks2[0, idx2] == toks1[0, idx1]
        idx2 += 1
    else:
        print('skipping pause token...')

w/o 1 <s> w 1 <s>
w/o 450 The w 450 The
skipping pause token...
w/o 4996 quick w 4996 quick
skipping pause token...
w/o 17354 brown w 17354 brown
skipping pause token...
w/o 1701 fo w 1701 fo
w/o 29916 x w 29916 x
w/o 432 j w 432 j
w/o 17204 umps w 17204 umps
w/o 975 over w 975 over
w/o 278 the w 278 the
w/o 17366 lazy w 17366 lazy
w/o 11203 dog w 11203 dog


In [10]:
out = model.generate(tokenizer.encode('The<|pause|> quick<|pause|> brown <|pause|> fox jumps over the lazy dog', return_tensors='pt'), max_length=50, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)

/dlabdata1/wendler/.rlllm/lib/python3.11/site-packages/transformers/generation/utils.py:1460: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [11]:
tokenizer.decode(out[0], skip_special_tokens=True)

'The quick brown fox jumps over the lazy dog ran the dog ran and the dog ran the dog ran over the dog and the dog ran over the dog and the dog ran over the dog and the dog and the dog and'

In [12]:
tokenizer.decode(out[0], skip_special_tokens=False)

'<s> The<|pause|> quick<|pause|> brown<|pause|> fox jumps over the lazy dog ran the dog ran and the dog ran the dog ran over the dog and the dog ran over the dog and the dog ran over the dog and the dog and the dog and'

In [13]:
surgeon.save(model, '/dlabscratch1/tmp/llama2_test')

In [14]:
surgeon2 = Llama2EmbeddingSurgeon.load('/dlabscratch1/tmp/llama2_test')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
surgeon2.get_surgeried_llama()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): EmbeddingWrapper(
      (old_embeddings): Embedding(32001, 4096)
      (new_embeddings): Embedding(32001, 4096)
    )
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post